In [1]:
import sys
import os
import pdb_numpy
from time import monotonic
import openmm
import openmm.app as app
from openmm import unit

sys.path.insert(0, '../src')

import SST2
import SST2.tools as tools
import SST2.rest2 as rest2
import SST2.sst2 as sst2

In [2]:
# Set system parameters
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3pfb.xml")
# Sim parameters
dt = 2 * unit.femtosecond
temperature = 300 * unit.kelvin
friction = 1 / unit.picoseconds

# Starting coordinates
PDB_PROT_PEP_SOL = '../src/SST2/tests/inputs/2HPL_equi_water.pdb'

pdb = app.PDBFile(PDB_PROT_PEP_SOL)
integrator = openmm.LangevinMiddleIntegrator(temperature, friction, dt)
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1 * unit.nanometers,
    constraints=app.HBonds,
)

simulation = tools.setup_simulation(system, pdb.positions, pdb.topology, integrator)

Created simulation


In [3]:
tot_steps = 200_000
freq_log_dcd = 10_000

OUT_DIR = "tmp_time_sim"
os.makedirs(OUT_DIR, exist_ok=True)

In [6]:
start = monotonic()

tools.simulate(
    simulation,
    pdb.topology,
    tot_steps,
    dt,
    f'{OUT_DIR}/simple_sim',
    save_step_log=freq_log_dcd,
    save_step_dcd=freq_log_dcd,
)

simulation_time = monotonic() - start
print(f'Classical MD simulation time took: {simulation_time:.1f} s.')

Timing 200000 steps of integration...
#"Step","Temperature (K)","Speed (ns/day)","Time Remaining"
10000,299.309156211985,0,--
20000,300.01401894878205,525,0:59
30000,302.75588629425624,535,0:54
40000,294.5370670624776,537,0:51
50000,298.26472392193295,535,0:48
60000,303.23304982666826,534,0:45
70000,302.2001968266329,533,0:42
80000,301.745168088392,532,0:38
90000,297.82267368408856,531,0:35
100000,306.1732232643654,531,0:32
110000,303.2852434125171,531,0:29
120000,300.70275368628575,531,0:26
130000,300.3726667788023,530,0:22
140000,301.9614100990066,530,0:19
150000,300.4111447163043,530,0:16
160000,302.1766898903606,530,0:13
170000,298.4437498032205,529,0:09
180000,299.3909930579146,529,0:06
190000,297.67901834147904,530,0:03
200000,300.6960195992583,530,0:00
200000 steps of 2.0 fs timestep (0.4 ns) took 65.6 s : 527.1 ns/day
Classical MD simulation time took: 66.0 s.


In [7]:
prot_pep_coor = pdb_numpy.Coor(PDB_PROT_PEP_SOL)
solute_indices = prot_pep_coor.get_index_select("chain B")

integrator_rest = openmm.LangevinMiddleIntegrator(temperature, friction, dt)

In [8]:
test = rest2.REST2(system, pdb, forcefield, solute_indices, integrator_rest)

Created simulation
Created simulation
Created simulation


In [10]:
start = monotonic()

rest2.run_rest2(
    test,
    f'{OUT_DIR}/rest2',
    tot_steps,
    dt,
    save_step_log=freq_log_dcd,
    save_step_dcd=freq_log_dcd,
    )

rest2_time = monotonic() - start
print(f'REST2 simulation time took: {rest2_time:.1f} s.')

Timing 200000 steps of integration...
#"Step","Temperature (K)","Speed (ns/day)","Time Remaining"
10000,303.862306600601,0,--
20000,301.6868878289856,443,1:10
30000,304.98395703487853,452,1:04
40000,294.8059511270358,455,1:00
50000,302.0885975802759,455,0:57
60000,297.75645233727846,456,0:53
70000,297.6008129030667,455,0:49
80000,304.2049272479925,456,0:45
90000,304.7937821690071,456,0:41
100000,299.4187656354094,454,0:38
110000,297.17202081628926,453,0:34
120000,301.1736766997813,453,0:30
130000,296.4070100250381,452,0:26
140000,298.91404406354144,451,0:22
150000,299.387067786804,450,0:19
160000,301.03302504688156,450,0:15
170000,299.62689296613075,450,0:11
180000,305.129179206585,450,0:07
190000,303.7281512821675,450,0:03
200000,299.7594563794769,449,0:00
200000 steps of 2.0 fs timestep (0.4 ns) took 77.1 s : 448.4 ns/day
REST2 simulation time took: 77.4 s.


In [13]:
print(f'REST2 slowdown is: {100 * (rest2_time - simulation_time) / simulation_time:.2f} %')

REST2 slowdown is: 17.32 %


In [14]:
ladder_num = tools.compute_ladder_num(
    f'{OUT_DIR}/rest2_rest2',
    300,
    500,
    sst2_score=True
)
ladder_num = 5

In [15]:
temp_list = tools.compute_temperature_list(
    minTemperature=300,
    maxTemperature=500,
    numTemperatures=ladder_num,
    refTemperature=300,
)
temp_list 

[Quantity(value=300, unit=kelvin),
 Quantity(value=340.8658099402498, unit=kelvin),
 Quantity(value=387.2983346207417, unit=kelvin),
 Quantity(value=440.05586839669667, unit=kelvin),
 Quantity(value=500.0, unit=kelvin)]

In [16]:
tempChangeInterval = int(2.0 * unit.picosecond / dt.in_units_of(unit.picosecond))
tempChangeInterval

1000

In [17]:
start = monotonic()

sst2.run_sst2(
    test,
    f'{OUT_DIR}/sst2',
    tot_steps,
    dt,
    temperatures=temp_list,
    ref_temp=temp_list[0],
    save_step_log=freq_log_dcd,
    save_step_dcd=freq_log_dcd,
    tempChangeInterval=tempChangeInterval,
)

sst2_time = monotonic() - start
print(f'REST2 simulation time took: {sst2_time:.1f} s.')

Timing 200000 steps of integration...
#"Step","Temperature (K)","Speed (ns/day)","Time Remaining"
10000,299.9186601244759,0,--
20000,301.8328023409841,430,1:12
30000,297.43700978885863,432,1:07
40000,297.32716679516534,435,1:03
50000,297.26862125164394,434,0:59
60000,302.1084810937353,433,0:55
70000,303.64253834443247,433,0:51
80000,301.26027710896005,433,0:47
90000,297.8018292739895,431,0:44
100000,299.88352241636846,432,0:40
110000,303.0778180713265,433,0:35
120000,304.1462239551278,431,0:32
130000,304.07698204724704,430,0:28
140000,295.9749748711528,430,0:24
150000,302.3123785541703,428,0:20
160000,298.6097699471747,428,0:16
170000,299.2449389184568,428,0:12
180000,302.2871530226596,428,0:08
190000,300.0405733733266,428,0:04
200000,302.4584289453144,429,0:00
200000 steps of 2.0 fs timestep (0.4 ns) took 80.7 s : 428.0 ns/day
REST2 simulation time took: 81.1 s.


In [18]:
print(f'SST2 slowdown is: {100 * (sst2_time - simulation_time) / simulation_time:.2f} %')

SST2 slowdown is: 22.95 %
